In [1]:
import xml.etree.ElementTree as ET
#Importing training data
x_train_tree = ET.parse('training.xml')
x_train_root=x_train_tree.getroot()
y_train_tree=ET.parse('AnnotationTraining.xml')
y_train_root=y_train_tree.getroot()
#Importing testing data
x_test_tree = ET.parse('testing.xml')
x_test_root=x_test_tree.getroot()
y_test_tree=ET.parse('AnnotationTesting.xml')
y_test_root=y_test_tree.getroot()

In [3]:
#testing the output format
print(x_train_root[0].text)


		hesitate in to giving is @aapleaks #aapsweep revenge should this statehood take way bjp not the #aapstorm best 
	


In [4]:
#testing the output format
print(y_train_root[0].text)


		en en en en en X X en en en en en en NE en en X en 
	


In [6]:
#below are the labels accepted and the model will be working on
req_labels=['mr','hi','te','ta','kn','gu','bn','ml','en']
#below are the labels used for sentence labeling.. 
#if any indian lang word used in the sentence...Label that sentence with that Indian lang 
indian_lang=['mr','hi','te','ta','kn','gu','bn','ml']

In [8]:
#process Data In list of list of tuple Format
def readDataSet(x_root,y_root):
    to_return = [] #[[(word,sentence_lang,word_label),.....],....]
    for i in range(len(x_root)): #iterate over each sentence
        words=x_root[i].text.split()
        labels=y_root[i].text.split()
        if len(words)!=len(labels): #error condition
            print("Excluded Error Line : ", i+1, " ", len(words), " ", len(labels))
        else:
            #indentify the indian lang used in the sentence
            lang_identify='en'
            for j in range(len(labels)):
                if labels[j] in indian_lang:
                    lang_identify=labels[j]
            #create a list [(word,sentence_lang,word_label),...] for each word of sentence 
            temp = []
            for j in range(len(labels)):
                if labels[j] in req_labels:
                    temp.append((words[j],lang_identify,labels[j]))
            to_return.append(temp)
    return to_return

In [10]:
#checking
print(x_train_root[0].text, "\n\n\n", y_train_root[0].text)


		hesitate in to giving is @aapleaks #aapsweep revenge should this statehood take way bjp not the #aapstorm best 
	 


 
		en en en en en X X en en en en en en NE en en X en 
	


In [14]:
#read train_data in required format
train_data=readDataSet(x_train_root,y_train_root)
#checking
print(train_data[1:2])

Excluded Error Line :  320   24   25
Excluded Error Line :  429   30   31
Excluded Error Line :  503   183   184
Excluded Error Line :  546   22   23
Excluded Error Line :  1201   57   58
Excluded Error Line :  1465   25   27
Excluded Error Line :  1788   47   48
Excluded Error Line :  1987   12   13
Excluded Error Line :  2083   22   24
Excluded Error Line :  2172   55   56
Excluded Error Line :  2372   21   22
Excluded Error Line :  2407   31   32
Excluded Error Line :  2488   50   51
[[('Tinbeku', 'kn', 'kn'), ('anstide', 'kn', 'kn')]]


In [15]:
#read test data in required format
test_data=readDataSet(x_test_root,y_test_root)
#checking
print(train_data[1:2])

Excluded Error Line :  380   130   131
Excluded Error Line :  391   45   47
Excluded Error Line :  471   23   24
Excluded Error Line :  538   34   35
Excluded Error Line :  660   23   24
Excluded Error Line :  724   7   8
Excluded Error Line :  728   22   23
[[('Tinbeku', 'kn', 'kn'), ('anstide', 'kn', 'kn')]]


In [16]:
#defining features at word level
def extract_features(sentence, position):
    features = []
    features.append('CurrentWord:'+sentence[position][0])
    if (position < len(sentence)-1):
        features.append('NextWord:'+sentence[position+1][0])
    if (position < len(sentence)-2):
        features.append('NextToNextWord:'+sentence[position+2][0])
    if (position > 0):
        features.append('PrevWord:'+sentence[position-1][0])
    if (position-1 > 0):
        features.append('PrevToPrevWord:'+sentence[position-2][0])
    if (position == 0):
        features.append('BOS')
    if (position == len(sentence)-1):
        features.append('EOS')
    features.append('CurrentLang:'+sentence[position][1])
    if (position < len(sentence)-1):
        features.append('NextLang:'+sentence[position+1][1])
    if (position < len(sentence)-2):
        features.append('NextToNextLang:'+sentence[position+2][1])
    if (position > 0):
        features.append('PrevLang:'+sentence[position-1][1])
    if (position-1 > 0):
        features.append('PrevToPrevLang:'+sentence[position-2][1])
    return features

In [19]:
#checking for 2st train sentence at position 1
print(extract_features(train_data[1],0))

['CurrentWord:Tinbeku', 'NextWord:anstide', 'BOS', 'CurrentLang:kn', 'NextLang:kn']


In [22]:
#checking for 4st train sentence at position 3
print(extract_features(train_data[3],2))

['CurrentWord:to', 'NextWord:hradaya', 'NextToNextWord:chhe', 'PrevWord:paase', 'PrevToPrevWord:tamaari', 'CurrentLang:gu', 'NextLang:gu', 'NextToNextLang:gu', 'PrevLang:gu', 'PrevToPrevLang:gu']


In [23]:
#extract features at sentence level
def extract_sentence_features(sentence):
    return [extract_features(sentence,pos) for pos in range(len(sentence))]
#extract features at dataset level
def extract_dataset_features(dataset):
    return [extract_sentence_features(dataset[sentence]) for sentence in range(len(dataset))]

In [24]:
#train_x => extract train dataset features 
train_x = extract_dataset_features(train_data)
#checking
print(len(train_x))
print(len(train_data))

2894
2894


In [25]:
#train_x => extract test dataset features 
test_x = extract_dataset_features(test_data)
#testing
print(len(test_x))
print(len(test_data))

785
785


In [26]:
#extract labels at sentence level
def extract_sentence_labels(sentence):
    return [sentence[pos][2] for pos in range(len(sentence))]
#extract labels at dataset level
def extract_dataset_labels(dataset):
    return [extract_sentence_labels(dataset[pos]) for pos in range(len(dataset))]

In [29]:
#train_y => Train dataset labels
train_y = extract_dataset_labels(train_data)
#checking
print(len(train_y))

2894


In [30]:
#test_y => Test dataset labels
test_y = extract_dataset_labels(test_data)
#checking
print(len(test_y))

785


In [34]:
#sklearn_crfsuite library used for training the model
import sklearn_crfsuite
crf = sklearn_crfsuite.CRF(algorithm='lbfgs',
    c1=0.1, #regularization parameter 1
    c2=0.1, #regularization parameter 2
    max_iterations=100, 
    all_possible_transitions=True,
    all_possible_states=True,)

In [35]:
#train the model
crf.fit(train_x,train_y)

CRF(algorithm='lbfgs', all_possible_states=True, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [36]:
#no of state features
print(len(crf.state_features_))
#no of transition features
print(len(crf.transition_features_))
#print the output labels
print(crf.classes_)

37236
81
['en', 'kn', 'ta', 'gu', 'bn', 'te', 'hi', 'mr', 'ml']


In [37]:
#print state features
print(crf.state_features_)

{('NextWord:in', 'en'): 0.742888, ('NextWord:in', 'kn'): -0.284362, ('NextWord:in', 'ta'): -1.002257, ('NextWord:in', 'bn'): -0.518012, ('NextWord:in', 'hi'): 0.721185, ('NextWord:in', 'ml'): -0.006059, ('NextToNextWord:to', 'en'): 0.169142, ('NextToNextWord:to', 'kn'): -1.062323, ('NextToNextWord:to', 'gu'): 0.216332, ('NextToNextWord:to', 'bn'): -0.449776, ('NextToNextWord:to', 'te'): 0.118956, ('NextToNextWord:to', 'hi'): 0.427772, ('NextToNextWord:to', 'mr'): -0.025369, ('NextToNextWord:to', 'ml'): -0.781238, ('BOS', 'kn'): 0.063436, ('BOS', 'ta'): 0.139047, ('BOS', 'gu'): -0.326605, ('BOS', 'bn'): 0.260228, ('BOS', 'te'): -0.095503, ('BOS', 'hi'): -0.226617, ('BOS', 'mr'): -0.230395, ('BOS', 'ml'): -0.21509, ('CurrentLang:en', 'en'): 5.355475, ('CurrentLang:en', 'kn'): -0.328745, ('CurrentLang:en', 'ta'): -0.439187, ('CurrentLang:en', 'bn'): -0.211206, ('CurrentLang:en', 'te'): -0.168064, ('CurrentLang:en', 'hi'): -0.002819, ('CurrentLang:en', 'mr'): -0.136044, ('CurrentLang:en', 

In [38]:
#for model evalution
from sklearn_crfsuite import metrics

In [39]:
#test the model with test_x data
test_pred = crf.predict(test_x)

In [40]:
#get the flat_f1_score 
metrics.flat_f1_score(test_y,test_pred,average='weighted',labels=crf.classes_)

0.857818575285415

In [77]:
#classify the report each label output wise i.e., language wise
print(metrics.flat_classification_report(
    test_y, test_pred, labels=req_labels, digits=3
))

              precision    recall  f1-score   support

          mr      0.906     0.998     0.950       454
          hi      0.895     0.913     0.904      1593
          te      0.654     1.000     0.791       519
          ta      0.820     0.989     0.896       543
          kn      0.840     0.998     0.912       598
          gu      0.464     1.000     0.634       185
          bn      0.899     0.992     0.943      1368
          ml      0.753     0.980     0.852       199
          en      0.944     0.700     0.804      3850

    accuracy                          0.859      9309
   macro avg      0.797     0.952     0.854      9309
weighted avg      0.883     0.859     0.858      9309



In [71]:
#for further indepth report
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix
from itertools import chain
def bio_classification_report(y_true, y_pred):
    """
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.

    Note that it requires scikit-learn 0.15+ (or a version from github master)
    to calculate averages properly!
    """
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))

    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}

    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )

#generate a detailed report
bio_classification_report(test_y,test_pred)

'              precision    recall  f1-score   support\n\n          bn       0.90      0.99      0.94      1368\n          en       0.94      0.70      0.80      3850\n          gu       0.46      1.00      0.63       185\n          hi       0.89      0.91      0.90      1593\n          kn       0.84      1.00      0.91       598\n          ml       0.75      0.98      0.85       199\n          mr       0.91      1.00      0.95       454\n          ta       0.82      0.99      0.90       543\n          te       0.65      1.00      0.79       519\n\n   micro avg       0.86      0.86      0.86      9309\n   macro avg       0.80      0.95      0.85      9309\nweighted avg       0.88      0.86      0.86      9309\n samples avg       0.86      0.86      0.86      9309\n'

In [45]:
#test true labels for manual checking
print(test_y)

[['en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en'], ['kn', 'en', 'kn', 'kn', 'kn', 'kn', 'en'], ['en', 'en', 'en', 'en', 'en'], ['te', 'te', 'te'], ['kn', 'kn'], ['bn', 'bn', 'en', 'bn', 'bn'], ['en'], ['hi', 'hi', 'hi', 'en', 'hi', 'hi', 'hi'], ['bn', 'bn', 'bn', 'bn', 'en', 'en', 'bn'], ['te', 'te', 'te', 'te', 'en', 'te', 'te', 'te', 'te'], ['en', 'en', 'hi', 'en', 'en', 'hi', 'hi', 'hi', 'hi', 'hi', 'hi', 'hi'], ['hi', 'hi', 'hi', 'hi', 'en', 'hi', 'hi'], ['en', 'en'], ['en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en'], ['kn', 'kn', 'kn', 'kn', 'en', 'kn', 'kn', 'en', 'kn', 'kn'], ['kn', 'kn', 'kn', 'kn', 'kn', 'kn', 'kn'], ['hi', 'en', 'en', 'hi', 'hi', 'en', 'hi', 'hi', 'hi', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'hi'], ['mr', 'mr', 'en', 'mr', 'mr', 'mr', 'mr', 'mr', 'mr', 'mr', 'mr', 'mr'], ['en', 'en', 'en', 'en', 'en', 'en', 'hi', 'en', 'en', 'en', 'en', 'en', 'en'], ['bn', 'bn', 'bn', 'bn', 'bn', 'bn'], ['bn', 'bn'], ['ml', 'm

In [47]:
#test predicted labels for manual checking
print(test_pred)

[['en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en'], ['kn', 'kn', 'kn', 'kn', 'kn', 'kn', 'kn'], ['en', 'en', 'en', 'en', 'en'], ['te', 'te', 'te'], ['kn', 'kn'], ['bn', 'bn', 'en', 'bn', 'bn'], ['en'], ['hi', 'hi', 'hi', 'hi', 'hi', 'hi', 'hi'], ['bn', 'bn', 'bn', 'bn', 'en', 'en', 'en'], ['te', 'te', 'te', 'te', 'te', 'te', 'te', 'te', 'te'], ['hi', 'en', 'hi', 'hi', 'hi', 'hi', 'hi', 'hi', 'hi', 'hi', 'hi', 'hi'], ['hi', 'hi', 'hi', 'hi', 'hi', 'hi', 'hi'], ['en', 'en'], ['en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en'], ['kn', 'kn', 'kn', 'kn', 'kn', 'kn', 'kn', 'kn', 'kn', 'kn'], ['kn', 'kn', 'kn', 'kn', 'kn', 'kn', 'kn'], ['en', 'en', 'en', 'en', 'en', 'en', 'hi', 'hi', 'hi', 'hi', 'hi', 'hi', 'hi', 'hi', 'hi', 'hi', 'hi'], ['mr', 'mr', 'mr', 'mr', 'mr', 'mr', 'mr', 'mr', 'mr', 'mr', 'mr', 'mr'], ['en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en'], ['bn', 'bn', 'bn', 'bn', 'bn', 'bn'], ['bn', 'bn'], ['ml', 'm

In [58]:
#Hyperparameter Optimization..To improve quality
#select regularization parameters using randomized search and 3-fold cross-validation.
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
# defining fixed parameters
crf1 = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True,
    all_possible_states=True
)
#regularization terms
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=req_labels)

# search for best parameters
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(train_x, train_y)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 10.9min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=CRF(algorithm='lbfgs', all_possible_states=True,
                                 all_possible_transitions=True, averaging=None,
                                 c=None, c1=0.1, c2=0.1,
                                 calibration_candidates=None,
                                 calibration_eta=None,
                                 calibration_max_trials=None,
                                 calibration_rate=None,
                                 calibration_samples=None, delta=None,
                                 epsilon=None, error_sensitive=None, ga...
                   param_distributions={'c1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f5495c9eb90>,
                                        'c2': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f5495ca2050>},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_sc

In [76]:
# crf = rs.best_estimator_
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.08234416258689053, 'c2': 0.00019857762304671588}
best CV score: 0.9196892235606356
model size: 0.94M


In [75]:
#using best parameters
crf = rs.best_estimator_
#predict with this new hyperparameterizied model
y_pred1 = crf.predict(test_x)
print(metrics.flat_classification_report(
    test_y, y_pred1, labels=req_labels, digits=3
))

              precision    recall  f1-score   support

          mr      0.926     0.998     0.961       454
          hi      0.892     0.937     0.914      1593
          te      0.725     0.996     0.839       519
          ta      0.853     0.974     0.910       543
          kn      0.851     0.995     0.918       598
          gu      0.538     0.989     0.697       185
          bn      0.903     0.988     0.944      1368
          ml      0.786     0.980     0.872       199
          en      0.953     0.750     0.840      3850

    accuracy                          0.881      9309
   macro avg      0.825     0.956     0.877      9309
weighted avg      0.897     0.881     0.880      9309



In [67]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
en     -> en      2.310407
te     -> te      2.137228
bn     -> bn      1.999997
kn     -> en      1.410982
ta     -> ta      1.388948
mr     -> mr      1.332878
hi     -> hi      1.259815
kn     -> kn      1.193724
en     -> ta      1.115451
en     -> kn      1.014284
gu     -> gu      0.981621
ml     -> ml      0.979981
ml     -> en      0.889390
ta     -> en      0.875540
en     -> ml      0.770846
mr     -> en      0.714980
en     -> te      0.703097
te     -> en      0.617430
gu     -> en      0.550340
en     -> gu      0.496805

Top unlikely transitions:
ta     -> en      0.875540
en     -> ml      0.770846
mr     -> en      0.714980
en     -> te      0.703097
te     -> en      0.617430
gu     -> en      0.550340
en     -> gu      0.496805
bn     -> en      0.403851
hi     -> en      0.271526
en     -> mr      0.265183
en     -> hi      0.248853
en     -> bn      0.156480
ta     -> gu      -0.000020
hi     -> mr      -0.031734
bn     -> gu      -0.090501
h

In [69]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

#the below ones were booting the model score
print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

#the below ones are responsible for major downfall of model score
print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:
11.336704 hi       CurrentWord:tab
10.613269 en       CurrentLang:en
10.430693 en       CurrentWord:phone
10.104478 en       CurrentWord:movie
10.067667 en       CurrentWord:So
10.021286 en       CurrentWord:post
9.643074 en       CurrentWord:song
9.638517 en       CurrentWord:cinema
9.589696 en       CurrentWord:bomb
9.457883 en       CurrentWord:family
9.316024 en       CurrentWord:news
9.251741 en       CurrentWord:school
9.203305 en       CurrentWord:police
9.167212 en       CurrentWord:film
9.148515 en       CurrentWord:thanks
9.130408 en       CurrentWord:job
9.084101 en       CurrentWord:try
9.072153 en       CurrentWord:seat
9.070033 en       CurrentWord:Post
9.048087 en       CurrentWord:but
9.023156 en       CurrentWord:miss
9.008509 en       CurrentWord:weekend
9.001515 en       CurrentWord:star
8.989168 en       CurrentWord:room
8.931762 en       CurrentWord:sir
8.921783 en       CurrentWord:full
8.889428 en       CurrentWord:serial
8.848303 en       CurrentWo

In [80]:
#RESULTS FROM ABOVE MODELS

In [78]:
#results
#model 1 report 
#classify the report each label output wise i.e., language wise
print(metrics.flat_classification_report(
    test_y, test_pred, labels=req_labels, digits=3
))

              precision    recall  f1-score   support

          mr      0.906     0.998     0.950       454
          hi      0.895     0.913     0.904      1593
          te      0.654     1.000     0.791       519
          ta      0.820     0.989     0.896       543
          kn      0.840     0.998     0.912       598
          gu      0.464     1.000     0.634       185
          bn      0.899     0.992     0.943      1368
          ml      0.753     0.980     0.852       199
          en      0.944     0.700     0.804      3850

    accuracy                          0.859      9309
   macro avg      0.797     0.952     0.854      9309
weighted avg      0.883     0.859     0.858      9309



In [79]:
#model 2 report (best model..by finding best paramenters)
#using best parameters
crf = rs.best_estimator_
#predict with this new hyperparameterizied model
y_pred1 = crf.predict(test_x)
print(metrics.flat_classification_report(
    test_y, y_pred1, labels=req_labels, digits=3
))

              precision    recall  f1-score   support

          mr      0.926     0.998     0.961       454
          hi      0.892     0.937     0.914      1593
          te      0.725     0.996     0.839       519
          ta      0.853     0.974     0.910       543
          kn      0.851     0.995     0.918       598
          gu      0.538     0.989     0.697       185
          bn      0.903     0.988     0.944      1368
          ml      0.786     0.980     0.872       199
          en      0.953     0.750     0.840      3850

    accuracy                          0.881      9309
   macro avg      0.825     0.956     0.877      9309
weighted avg      0.897     0.881     0.880      9309



In [81]:
#Answer to the question "Number of true state features and tranistion features?"..
#The documenation gives the answer to the question.. 
#Extract from doc. for the same is gievn below..

"""
1> all_possible_states (bool, optional (default=False)) – 
Specify whether CRFsuite generates state features that do not even occur in the training data 
(i.e., negative state features). 
When True, CRFsuite generates state features that associate all of possible combinations between attributes 
and labels.
Suppose that the numbers of attributes and labels are A and L respectively, 
this function will generate (A * L) features. 
Enabling this function may improve the labeling accuracy because the CRF model can learn the condition where 
an item is not predicted to its reference label. 
However, this function may also increase the number of features and slow down the training process drastically. 
This function is disabled by default.


2> all_possible_transitions (bool, optional (default=False)) – 
Specify whether CRFsuite generates transition features that do not even occur in the training data 
(i.e., negative transition features). 
When True, CRFsuite generates transition features that associate all of possible label pairs. 
Suppose that the number of labels in the training data is L, 
this function will generate (L * L) transition features. 
This function is disabled by default.


3> c1 (float, optional (default=0)) –
The coefficient for L1 regularization. 
If a non-zero value is specified, CRFsuite switches to the Orthant-Wise Limited-memory Quasi-Newton (OWL-QN) method. 
The default value is zero (no L1 regularization).


4> c2 (float, optional (default=1.0)) – 
The coefficient for L2 regularization.

"""
print()